# feature correlations

In [1]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

In [2]:
#define features path
dreamspath=datapath+"/DREAMS"
featurespath=dreamspath+"/features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [3]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [4]:
signalsMetadata

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS
5,excerpt6.txt,CZ-A1,0006,1800,200,DREAMS
6,excerpt7.txt,CZ-A1,0007,1800,200,DREAMS
7,excerpt8.txt,CZ-A1,0008,1800,200,DREAMS


## Load features

In [5]:
windowDurations

[0.5, 1, 1.5, 2]

In [6]:
featureDf=pd.DataFrame()

durationFolders=os.listdir(dreamspath+'/features/200fs')
for duration in durationFolders:
    subjectFolders=os.listdir(dreamspath+'/features/200fs/'+duration)
    for subject in subjectFolders:
        aux=os.listdir(dreamspath+'/features/200fs/'+duration+'/'+subject)
        thisFeatureDf=pd.DataFrame({
            'filename':aux
        })
        featureDf=pd.concat([featureDf,thisFeatureDf],axis=0)

In [7]:
featureDf

,filename
0,0.5_0001_hjortActivity_alpha.fd
1,0.5_0001_hjortActivity_beta1.fd
2,0.5_0001_hjortActivity_beta2.fd
3,0.5_0001_hjortActivity_broadband.fd
4,0.5_0001_hjortActivity_delta1.fd
...,...
28,2_0008_petrosian_delta1.fd
29,2_0008_petrosian_delta2.fd
30,2_0008_petrosian_sigma.fd
31,2_0008_petrosian_theta.fd


In [8]:
featureDf['window']=featureDf.apply(lambda row:
                                    row.filename.split('_')[0],axis=1)
featureDf['subjectId']=featureDf.apply(lambda row:
                                    row.filename.split('_')[1],axis=1)
featureDf['characteristic']=featureDf.apply(lambda row:
                                    row.filename.split('_')[2],axis=1)
featureDf['bandName']=featureDf.apply(lambda row:
                                    row.filename.split('_')[3].split('.')[0],axis=1)

In [9]:
featureDf

,filename,window,subjectId,characteristic,bandName
0,0.5_0001_hjortActivity_alpha.fd,0.5,0001,hjortActivity,alpha
1,0.5_0001_hjortActivity_beta1.fd,0.5,0001,hjortActivity,beta1
2,0.5_0001_hjortActivity_beta2.fd,0.5,0001,hjortActivity,beta2
3,0.5_0001_hjortActivity_broadband.fd,0.5,0001,hjortActivity,broadband
4,0.5_0001_hjortActivity_delta1.fd,0.5,0001,hjortActivity,delta1
...,...,...,...,...,...
28,2_0008_petrosian_delta1.fd,2,0008,petrosian,delta1
29,2_0008_petrosian_delta2.fd,2,0008,petrosian,delta2
30,2_0008_petrosian_sigma.fd,2,0008,petrosian,sigma
31,2_0008_petrosian_theta.fd,2,0008,petrosian,theta


In [10]:
uniqueFeature=featureDf[['window','characteristic','bandName']].drop_duplicates(ignore_index=True)

In [11]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


In [12]:
uniqueFeature

,window,characteristic,bandName
0,0.5,hjortActivity,alpha
1,0.5,hjortActivity,beta1
2,0.5,hjortActivity,beta2
3,0.5,hjortActivity,broadband
4,0.5,hjortActivity,delta1
...,...,...,...
127,2,petrosian,delta1
128,2,petrosian,delta2
129,2,petrosian,sigma
130,2,petrosian,theta


## load some features

In [17]:
hjortActivity_001_05=loadFeature(0.5,'0001','hjortActivity','sigma',200,datapath+"/DREAMS"+"/features")
hjortActivity_001_1=loadFeature(1,'0001','hjortActivity','sigma',200,datapath+"/DREAMS"+"/features")
hjortActivity_001_15=loadFeature(1.5,'0001','hjortActivity','sigma',200,datapath+"/DREAMS"+"/features")
hjortActivity_001_2=loadFeature(2,'0001','hjortActivity','sigma',200,datapath+"/DREAMS"+"/features")

In [21]:
petrosian_001_05=loadFeature(0.5,'0001','petrosian','broadband',200,datapath+"/DREAMS"+"/features")
petrosian_001_1=loadFeature(1,'0001','petrosian','broadband',200,datapath+"/DREAMS"+"/features")
petrosian_001_15=loadFeature(1.5,'0001','petrosian','broadband',200,datapath+"/DREAMS"+"/features")
petrosian_001_2=loadFeature(2,'0001','petrosian','broadband',200,datapath+"/DREAMS"+"/features")

In [22]:
np.corrcoef(hjortActivity_001_05,hjortActivity_001_1)

array([[1.        , 0.86515837],
       [0.86515837, 1.        ]])

In [23]:
np.corrcoef(hjortActivity_001_05,hjortActivity_001_15)

array([[1.        , 0.75157051],
       [0.75157051, 1.        ]])

In [24]:
np.corrcoef(hjortActivity_001_05,hjortActivity_001_2)

array([[1.        , 0.70612772],
       [0.70612772, 1.        ]])

In [25]:
np.corrcoef(hjortActivity_001_05,petrosian_001_05)

array([[ 1.        , -0.03812989],
       [-0.03812989,  1.        ]])

In [26]:
np.corrcoef(hjortActivity_001_05,petrosian_001_1)

array([[ 1.        , -0.05378161],
       [-0.05378161,  1.        ]])

In [27]:
np.corrcoef(hjortActivity_001_05,petrosian_001_15)

array([[ 1.        , -0.06005247],
       [-0.06005247,  1.        ]])

In [28]:
np.corrcoef(hjortActivity_001_05,petrosian_001_2)

array([[ 1.        , -0.06216819],
       [-0.06216819,  1.        ]])

In [29]:
np.corrcoef(hjortActivity_001_1,hjortActivity_001_15)

array([[1.        , 0.93222438],
       [0.93222438, 1.        ]])

In [30]:
np.corrcoef(hjortActivity_001_1,hjortActivity_001_2)

array([[1.       , 0.8562394],
       [0.8562394, 1.       ]])

In [31]:
np.corrcoef(hjortActivity_001_1,petrosian_001_05)

array([[ 1.        , -0.06056732],
       [-0.06056732,  1.        ]])

In [32]:
np.corrcoef(hjortActivity_001_1,petrosian_001_1)

array([[ 1.        , -0.06693187],
       [-0.06693187,  1.        ]])

In [33]:
np.corrcoef(hjortActivity_001_1,petrosian_001_15)

array([[ 1.        , -0.07350426],
       [-0.07350426,  1.        ]])

In [34]:
np.corrcoef(hjortActivity_001_1,petrosian_001_2)

array([[ 1.       , -0.0789152],
       [-0.0789152,  1.       ]])

In [36]:
np.corrcoef(hjortActivity_001_15,petrosian_001_05)

array([[ 1.        , -0.07405574],
       [-0.07405574,  1.        ]])

In [37]:
np.corrcoef(hjortActivity_001_15,petrosian_001_1)

array([[ 1.       , -0.0806171],
       [-0.0806171,  1.       ]])

In [38]:
np.corrcoef(hjortActivity_001_15,petrosian_001_15)

array([[ 1.        , -0.08508826],
       [-0.08508826,  1.        ]])

In [39]:
np.corrcoef(hjortActivity_001_15,petrosian_001_2)

array([[ 1.        , -0.09030762],
       [-0.09030762,  1.        ]])

In [40]:
np.corrcoef(hjortActivity_001_2,petrosian_001_05)

array([[ 1.        , -0.08358658],
       [-0.08358658,  1.        ]])

In [41]:
np.corrcoef(hjortActivity_001_2,petrosian_001_1)

array([[ 1.        , -0.09437706],
       [-0.09437706,  1.        ]])

In [42]:
np.corrcoef(hjortActivity_001_2,petrosian_001_15)

array([[ 1.        , -0.09853607],
       [-0.09853607,  1.        ]])

In [43]:
np.corrcoef(hjortActivity_001_2,petrosian_001_2)

array([[ 1.        , -0.10190022],
       [-0.10190022,  1.        ]])

In [44]:
np.corrcoef(petrosian_001_05,petrosian_001_1)

array([[1.        , 0.90891226],
       [0.90891226, 1.        ]])

In [45]:
np.corrcoef(petrosian_001_05,petrosian_001_15)

array([[1.        , 0.87217192],
       [0.87217192, 1.        ]])

In [46]:
np.corrcoef(petrosian_001_05,petrosian_001_2)

array([[1.        , 0.85545321],
       [0.85545321, 1.        ]])

In [47]:
np.corrcoef(petrosian_001_1,petrosian_001_15)

array([[1.        , 0.96422613],
       [0.96422613, 1.        ]])

In [48]:
np.corrcoef(petrosian_001_1,petrosian_001_2)

array([[1.        , 0.94403281],
       [0.94403281, 1.        ]])

In [49]:
np.corrcoef(petrosian_001_15,petrosian_001_2)

array([[1.       , 0.9801848],
       [0.9801848, 1.       ]])